# Training DLite

This notebook fine-tunes the smallest GPT2 model on the `databricks-dolly-15k` dataset, thus training the `dlite-v2-124m` model.

In [ ]:
%pip install -r requirements.txt
! sh setup.sh

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from train.utils import create_response, load_model_and_tokenizer, DEFAULT_MAX_LENGTH, DATASET
import datetime as dt
import mlflow
import os

In [ ]:
# Assign the output directory and create it
output_dir = 'dlite-v2-124m'
print(f'Output path: {output_dir}')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

epochs = 10
train_batch_size = 16
eval_batch_size = 16
lr = 1e-5
model_id = 'gpt2'
deepspeed_config = './deepspeed_config.json'
max_length = DEFAULT_MAX_LENGTH
dataset = DATASET

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '{output_dir}/runs'

In [ ]:
# Run the train script
! deepspeed ./train/train.py \
    {output_dir} \
    -e {epochs} \
    --train-batch-size {train_batch_size} \
    --eval-batch-size {eval_batch_size} \
    --lr {lr} \
    --gradient-checkpointing \
    --cuda \
    -m {model_id} \
    --deepspeed {deepspeed_config} \
    --local_rank True \
    --fp16 \
    --max-length {max_length} \
    --dataset {dataset}


In [ ]:
# Load the saved model and tokenizer
model, tokenizer = load_model_and_tokenizer(output_dir)

# Set of example prompts
eval_prompts = [
    'Write a tweet announcing DLite, a large language model from AI Squared.',
    'Write a poem.',
    'How many legs do dogs typically have?',
    'Explain the concept of deep learning.'
]

# Ask DLite to answer the prompts listed above
for prompt in eval_prompts:
    response = create_response(prompt, model = model, tokenizer = tokenizer, device = 'cuda')
    print(f'Prompt: {prompt}\n\n{response}\n\n-----------\n')

# Log the model with mlflow
with mlflow.start_run():
    mlflow.log_params(
        {
            'base_model': model_id,
            'dataset': dataset,
            'epochs': epochs,
            'lr': lr,
            'train batch size': train_batch_size,
            'eval batch size': eval_batch_size,
            'max length': max_length
        }
    )
    mlflow.transformers.log_model(
        transformers_model = {
            'model': model,
            'tokenizer': tokenizer
        },
        artifact_path = output_dir,
        task = 'text-generation'
    )